## Introduction

In this example, we will using a simply long hold strategy to demostrate the usage of different rebalancing methods.


We are using a simple version of Ray Dalio's All Weather Portfolio as an example. Ray Dalio's All Weather portfolio consists of the following assets and target weights:

- US Stocks (40%)
- Long Term US Bonds (30%)
- Intermediate US Bonds (15%)
- Gold (7.5%)
- Commodities (7.5%)


In this example, we will simplify it to US Stock Index Fund, Bond ETF and Gold ETF with the following target weights:
- US Stock Index Fund (50%)
- Bond ETF (40%)
- Gold ETF (10%)



## Import Data

In [1]:
from tiportfolio.helpers.data import Alpaca
import dotenv

dotenv.load_dotenv()

import os
alpaca = Alpaca(os.environ['ALPACA_API_KEY'], os.environ['ALPACA_API_SECRET'])

In [2]:
# use apple to test long hold strategy

apple_df = alpaca.query('AAPL', '2019-01-01', '2020-12-31', '1D')
# use date column as index
# apple_df.set_index('date', inplace=True)

Loading bar data...
Loaded bar data: 0:00:01 



In [3]:
apple_df.index

RangeIndex(start=0, stop=504, step=1)

In [4]:
from tiportfolio.strategy_library.trading.long_hold import LongHold
long_hold_strategy = LongHold(symbol="AAPL", prices=apple_df)
long_hold_strategy.before_all()

for step in long_hold_strategy.all_steps:
    long_hold_strategy.execute(step)



long_hold_strategy.after_all()[0]

,symbol,open,high,low,close,volume,vwap,signal,pnl,value,cumulative_pnl,cummax,drawdown,max_drawdown,mar_ratio
date,,,,,,,,,,,,,,,
2019-01-02 00:00:00-05:00,AAPL,36.82,37.76,36.66,37.54,158163180.0,37.33,1.0,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
2019-01-03 00:00:00-05:00,AAPL,34.23,34.64,33.76,33.80,387734780.0,34.11,1.0,-0.099627,0.900373,-0.099627,1.000000,-0.099627,-0.099627,-1.000000
2019-01-04 00:00:00-05:00,AAPL,34.36,35.31,34.18,35.24,247113388.0,34.92,1.0,0.042604,0.938732,-0.061268,1.000000,-0.061268,-0.099627,-0.614973
2019-01-07 00:00:00-05:00,AAPL,35.35,35.38,34.68,35.17,230235968.0,35.04,1.0,-0.001986,0.936867,-0.063133,1.000000,-0.063133,-0.099627,-0.633690
2019-01-08 00:00:00-05:00,AAPL,35.55,36.09,35.31,35.84,173051260.0,35.69,1.0,0.019050,0.954715,-0.045285,1.000000,-0.045285,-0.099627,-0.454545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23 00:00:00-05:00,AAPL,128.66,128.93,127.32,127.50,96689113.0,128.09,1.0,-0.006932,3.396377,2.396377,3.473895,-0.022314,-0.314253,7.625625
2020-12-24 00:00:00-05:00,AAPL,127.85,129.93,127.63,128.48,56888726.0,128.86,1.0,0.007686,3.422483,2.422483,3.473895,-0.014799,-0.314253,7.708696
2020-12-28 00:00:00-05:00,AAPL,130.45,133.71,129.98,133.08,131183580.0,132.28,1.0,0.035803,3.545019,2.545019,3.545019,0.000000,-0.314253,8.098624


In [5]:
long_hold_strategy.after_all()[1]

{'final_value': 3.467767714437935,
 'total_return': 2.467767714437935,
 'max_drawdown': -0.314253222137983,
 'mar_ratio': 7.852800037017224}

In [6]:
from tiportfolio.portfolio.allocation.allocation import PortfolioConfig
from tiportfolio.portfolio.allocation.frequency_based_allocation import RebalanceFrequency
from tiportfolio.strategy_library.allocation.fix_percentage import FixPercentageFrequencyBasedAllocation

config:PortfolioConfig = {
    "fees_config": {
        "commission": 0.0001,
        "slippage": 0.0,
    },
    "risk_free_rate": 0.0,
    "initial_capital": 100_000,
    "market_name": "NYSE",
}

long_hold_strategy = LongHold(symbol="AAPL", prices=apple_df)

fix_percentage_allocation = FixPercentageFrequencyBasedAllocation(
    config=config,
    strategies=[long_hold_strategy],
    allocation_percentages=[1.0],
    rebalance_frequency=RebalanceFrequency.mid_of_month,
    market_name="NYSE",
)

In [7]:
fix_percentage_allocation.walk_forward()

In [10]:
fix_percentage_allocation.strategy_quantity_map

{'LongHold - AAPL': 0.0}

In [9]:
fix_percentage_allocation.portfolio_df

,,signal,open,high,low,close,quantity,value,fees,cost_basis
datetime,strategy_unique_name,,,,,,,,,
